## Generatic synthetic data using a NLME model


In [1]:
import sys
import numpy as np
import pandas as pd
from IPython.display import display

sys.path.append("../")
from vpop_calibration.model.ode import OdeModel
from vpop_calibration.data_generation import simulate_dataset_from_omega

%load_ext autoreload
%autoreload 2

In [2]:
def equations(t, y, k_a, k_12, k_21, k_el):
    # y[0] is A_absorption, y[1] is A_central, y[2] is A_peripheral
    A_absorption, A_central, A_peripheral = y[0], y[1], y[2]
    dA_absorption_dt = -k_a * A_absorption
    dA_central_dt = (
        k_a * A_absorption + k_21 * A_peripheral - k_12 * A_central - k_el * A_central
    )
    dA_peripheral_dt = k_12 * A_central - k_21 * A_peripheral

    ydot = [dA_absorption_dt, dA_central_dt, dA_peripheral_dt]
    return ydot


variable_names = ["A0", "A1", "A2"]
parameter_names = ["k_a", "k_12", "k_21", "k_el"]

pk_two_compartments_abs_model = OdeModel(equations, variable_names, parameter_names)

In [3]:
# Create an NLME model
# First the structural model
# We need an ODE model, initial conditions and time steps

ode_model = pk_two_compartments_abs_model
print(ode_model.variable_names)

protocol_design = pd.DataFrame(
    {"protocol_arm": ["arm-A", "arm-B"], "k_el": [0.5, 5.2]}
)  # this is just a dummy design
time_span = (0, 24)
nb_steps = 20

# For each output and for each patient, give a list of time steps to be simulated
time_steps = np.linspace(time_span[0], time_span[1], nb_steps).tolist()

initial_conditions = np.array([10.0, 0.0, 0.0])

# Parameter definitions
init_log_MI = {"k_a": 0.5}
init_log_PDU = {
    "k_12": {"mean": -1, "sd": 0.25},
    "k_21": {"mean": 0, "sd": 0.25},
}
error_model_type = "additive"
init_res_var = [0.05, 0.02, 0.1]
covariate_map = {
    "k_12": {
        "foo": {"coef": "cov_foo_k12", "value": 0.5},
        "bar": {"coef": "cov_bar_k12", "value": 1.5},
    },
    "k_21": {},
}  # list for each PDU, which covariate influences it, and the name of the coefficient

patient_covariates = pd.DataFrame(
    {
        "id": ["a", "b", "c", "d"],
        "protocol_arm": ["arm-A", "arm-A", "arm-B", "arm-B"],
        "foo": [1, 2, 3, 4],
        "bar": [1, 1, 1, 1],
    }
)

['A0', 'A1', 'A2']


In [4]:
obs_df = simulate_dataset_from_omega(
    ode_model,
    protocol_design,
    time_steps,
    initial_conditions,
    init_log_MI,
    init_log_PDU,
    error_model_type,
    init_res_var,
    covariate_map,
    patient_covariates,
)

display(obs_df)

Successfully loaded 2 covariates:
['bar', 'foo']


,k_21,k_12,k_a,id,protocol_arm,foo,bar,output_name,time,A0_0,A1_0,A2_0,k_el,value
0,2.881573,0.782159,1.648721,a,arm-A,1,1,A0,0.000000,10.0,0.0,0.0,0.5,1.000000e+01
1,2.881573,0.782159,1.648721,a,arm-A,1,1,A0,1.263158,10.0,0.0,0.0,0.5,1.246065e+00
2,2.881573,0.782159,1.648721,a,arm-A,1,1,A0,2.526316,10.0,0.0,0.0,0.5,1.552683e-01
3,2.881573,0.782159,1.648721,a,arm-A,1,1,A0,3.789474,10.0,0.0,0.0,0.5,1.934739e-02
4,2.881573,0.782159,1.648721,a,arm-A,1,1,A0,5.052632,10.0,0.0,0.0,0.5,2.410691e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,8.824148,1.137306,1.648721,d,arm-B,4,1,A2,18.947368,10.0,0.0,0.0,5.2,-3.448874e-10
236,8.824148,1.137306,1.648721,d,arm-B,4,1,A2,20.210526,10.0,0.0,0.0,5.2,-2.679512e-10
237,8.824148,1.137306,1.648721,d,arm-B,4,1,A2,21.473684,10.0,0.0,0.0,5.2,-1.910149e-10
238,8.824148,1.137306,1.648721,d,arm-B,4,1,A2,22.736842,10.0,0.0,0.0,5.2,-1.140786e-10
